## Research commodity options calendar spreads, especially in ES
### Try to optimize rolling portfolio protection

In [1]:
import zipfile
import glob
import pandas as pd
import numpy as np

from argparse import ArgumentParser
from argparse import RawDescriptionHelpFormatter
import sys
import os
if  not './' in sys.path:
    sys.path.append('./')
if  not '../' in sys.path:
    sys.path.append('../')

from barchartacs import build_db
from barchartacs import db_info
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.tools as tls
from plotly.graph_objs.layout import Font,Margin
from IPython import display

import datetime
import io
from tqdm import tqdm,tqdm_notebook
from barchartacs import pg_pandas as pg
import mibian
import py_vollib
import importlib
from py_vollib import black
from py_vollib.black import implied_volatility
import ipdb
import traceback
import pandas_datareader.data as pdr

# importlib.reload(build_db)

/Users/bperlman1/Virtualenvs3/dashrisk4/lib/python3.6/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



## IF YOU WANT TO SEE WARNINGS, COMMENT THIS OUT

In [2]:
import warnings
warnings.filterwarnings("ignore")

### important global variables

In [94]:

DEBUG_IT=False
opttab = 'sec_schema.options_table'
futtab = 'sec_schema.underlying_table'
SYMBOL_TO_RESEARCH = 'ES'
STRIKE_DIVISORS = {}

# df_expiry_dates_additions = pd.read_csv('df_expiry_dates_additions.csv')
df_expiry_dates_additions = pd.read_csv('live_option_expirations.csv')


#### get all contracts in the options database

In [5]:
pga = db_info.get_db_info()
print(f"futtab max date: {pga.get_sql(f'select max(settle_date) from {futtab}')}")
print(f"opttab max date: {pga.get_sql(f'select max(settle_date) from {opttab}')}")


  sec_db
futtab max date:         max
0  20200310
opttab max date:         max
0  20200310


In [4]:
def plotly_plot(df_in,x_column,plot_title=None,
                y_left_label=None,y_right_label=None,
                bar_plot=False,figsize=(16,10),
                number_of_ticks_display=20,
                yaxis2_cols=None,
                x_value_labels=None):
    ya2c = [] if yaxis2_cols is None else yaxis2_cols
    ycols = [c for c in df_in.columns.values if c != x_column]
    # create tdvals, which will have x axis labels
    td = list(df_in[x_column]) 
    nt = len(df_in)-1 if number_of_ticks_display > len(df_in) else number_of_ticks_display
    spacing = len(td)//nt
    tdvals = td[::spacing]
    tdtext = tdvals
    if x_value_labels is not None:
        tdtext = [x_value_labels[i] for i in tdvals]
    
    # create data for graph
    data = []
    # iterate through all ycols to append to data that gets passed to go.Figure
    for ycol in ycols:
        if bar_plot:
            b = go.Bar(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        else:
            b = go.Scatter(x=td,y=df_in[ycol],name=ycol,yaxis='y' if ycol not in ya2c else 'y2')
        data.append(b)

    # create a layout
    layout = go.Layout(
        title=plot_title,
        xaxis=dict(
            ticktext=tdtext,
            tickvals=tdvals,
            tickangle=45,
            type='category'),
        yaxis=dict(
            title='y main' if y_left_label is None else y_left_label
        ),
        yaxis2=dict(
            title='y alt' if y_right_label is None else y_right_label,
            overlaying='y',
            side='right'),
        margin=Margin(
            b=100
        )        
    )

    fig = go.Figure(data=data,layout=layout)
    fig.update_layout(
        title={
            'text': plot_title,
            'y':0.9,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'})
    return fig

def plotly_shaded_rectangles(beg_end_date_tuple_list,fig):
    ld_shapes = []
    for beg_end_date_tuple in beg_end_date_tuple_list:
        ld_beg = beg_end_date_tuple[0]
        ld_end = beg_end_date_tuple[1]
        ld_shape = dict(
            type="rect",
            # x-reference is assigned to the x-values
            xref="x",
            # y-reference is assigned to the plot paper [0,1]
            yref="paper",
            x0=ld_beg[i],
            y0=0,
            x1=ld_end[i],
            y1=1,
            fillcolor="LightSalmon",
            opacity=0.5,
            layer="below",
            line_width=0,
        )
        ld_shapes.append(ld_shape)

    fig.update_layout(shapes=ld_shapes)
    return fig

In [104]:
def _next_monthyear_code(contract):
    code_val = contract[-3]
    code_num = DICT_MONTH_NUMS[code_val]
    y = int(contract[-2:])
    if code_num+1>12:
        next_code_num = 1
        next_y = y + 1
    else:
        next_code_num = code_num+1
        next_y = y
    next_code_val = MONTH_CODES[next_code_num-1]
    next_contract = contract[0:-3] + next_code_val + '%02d' %(next_y)
    return next_contract

def get_postgres_data(contract,strike_divisor=None):
    '''
    Get options and underlying data for ONLY ONE CONTRACT
    '''
    osql = f"select * from {opttab} where symbol='{contract}';"
    dfo = pga.get_sql(osql)
    if len(dfo)<10:
        e = f'''
        get_postgres_data ERROR: not enough option data for contract {contract} 
        '''
        raise ValueError(e)
    num_settle_days = len(dfo.settle_date.unique())
    u_contract = contract
    for i in range(12):
        usql = f"select * from {futtab} where symbol='{u_contract}';"
        dfu = pga.get_sql(usql)
        if len(dfu) < num_settle_days:
            u_contract = _next_monthyear_code(u_contract)
            print(f'trying contract {u_contract}')
        else:
            break

    if len(dfu)< num_settle_days:
        e = f'''
        get_postgres_data ERROR: not enough underlying days found for options contract {contract} 
        where len(underlying) = {len(dfu)} and num_settle_days = {num_settle_days}
        '''
        raise ValueError(e)
    # Merge options and futures data
    dfu = dfu.rename(columns={'symbol':'u_symbol'})
    df = dfo.merge(dfu,how='inner',on=['settle_date'])
    # Get options expiration dates
    df_expiry_dates = dfo[['symbol','settle_date']].groupby('symbol',as_index=False).max()
    df_additions = df_expiry_dates_additions[df_expiry_dates_additions.symbol==contract]
    df_additions = df_additions[['symbol','yyyymmdd_option']].rename(columns={'yyyymmdd_option':'settle_date'})
    additional_symbols = df_additions.symbol.values
    df_expiry_dates = df_expiry_dates[~df_expiry_dates.symbol.isin(additional_symbols)]
    df_expiry_dates = df_expiry_dates.append(df_additions).sort_values('symbol').copy()
    if strike_divisor is not None:
        df.strike = df.strike/strike_divisor
    return df,df_expiry_dates

### analyse option volumes by day, to see largest volumes

In [15]:
np.arange(17,20,1)

array([17, 18, 19])

In [56]:
#sql
commod = 'ES'
m = 'HUMZUHZM'
y = np.array([np.array([i,i,i,i,i,i+1,i,i+1]) for i in np.arange(17,20,1)])


myy_pairs = np.array(
                [np.array([f'{m[j]}{y[i][j]}' for j in range(len(m))]
            ) for i in range(len(y))])
myy_pairs = myy_pairs.reshape(-1).reshape(-1,2)
strike = 2500
for mp in myy_pairs:
    leg_front = f'{commod}{mp[0]}'
    leg_back = f'{commod}{mp[1]}'
    sql_select_options = f"""
    select o.settle_date,o.symbol,o.strike,o.pc,o.close o_close from {opttab} o 
    where symbol in ('{leg_front}','{leg_back}') 
    and o.strike={strike}
    join {futtab} f on o.symbol = f.symbol and o.settle_date = f.settle_date
    order by o.settle_date,o.symbol
    """

In [87]:
myy_pairs

array([['H17', 'U17'],
       ['M17', 'Z17'],
       ['U17', 'H18'],
       ['Z17', 'M18'],
       ['H18', 'U18'],
       ['M18', 'Z18'],
       ['U18', 'H19'],
       ['Z18', 'M19'],
       ['H19', 'U19'],
       ['M19', 'Z19'],
       ['U19', 'H20'],
       ['Z19', 'M20']], dtype='<U3')

In [92]:
mp = ['H19', 'U19']#myy_pairs[9]
strike = 2750.0
pc = 'P'
leg_front = f'{commod}{mp[0]}'
leg_back = f'{commod}{mp[1]}'
sql_select_options = f"""
select o.settle_date,o.symbol,o.strike,o.pc,o.close o_close,f.close f_close from {opttab} o 
join {futtab} f on o.symbol = f.symbol and o.settle_date = f.settle_date
where o.symbol in ('{leg_front}','{leg_back}') 
and o.strike={strike} and o.pc = '{pc}'
order by o.settle_date,o.symbol
"""
df_opt_pair = pga.get_sql(sql_select_options)
df_front = df_opt_pair[df_opt_pair.symbol==leg_front]
df_front = df_front.rename(columns={'symbol':'fr_symbol','o_close':'fr_opt_pr','f_close':'fr_fut_pr'})
df_back = df_opt_pair[df_opt_pair.symbol==leg_back]
df_back = df_back.rename(columns={'symbol':'bk_symbol','o_close':'bk_opt_pr','f_close':'bk_fut_pr'})
merge_cols = ['settle_date','strike','pc']
df_both = df_front.merge(df_back,how='inner',on=merge_cols)
df_both['fut_spread'] = df_both.bk_fut_pr - df_both.fr_fut_pr
df_both['opt_spread'] = df_both.bk_opt_pr - df_both.fr_opt_pr
df_both 

,settle_date,fr_symbol,strike,pc,fr_opt_pr,fr_fut_pr,bk_symbol,bk_opt_pr,bk_fut_pr,fut_spread,opt_spread
0,20180924,ESH19,2750.0,P,50.50,2934.00,ESU19,99.75,2951.50,17.50,49.25
1,20180925,ESH19,2750.0,P,51.50,2930.00,ESU19,100.25,2950.00,20.00,48.75
2,20180926,ESH19,2750.0,P,53.50,2920.25,ESU19,103.00,2940.00,19.75,49.50
3,20180927,ESH19,2750.0,P,51.00,2928.50,ESU19,100.00,2948.50,20.00,49.00
4,20180928,ESH19,2750.0,P,50.75,2927.50,ESU19,100.25,2948.50,21.00,49.50
...,...,...,...,...,...,...,...,...,...,...,...
114,20190311,ESH19,2750.0,P,4.20,2784.00,ESU19,99.00,2793.75,9.75,94.80
115,20190312,ESH19,2750.0,P,1.90,2792.00,ESU19,94.75,2801.50,9.50,92.85
116,20190313,ESH19,2750.0,P,0.40,2814.50,ESU19,86.00,2824.50,10.00,85.60
117,20190314,ESH19,2750.0,P,0.30,2807.25,ESU19,87.75,2817.00,9.75,87.45


In [93]:
graph_cols = ['settle_date','fr_fut_pr','fut_spread','opt_spread']
iplot(plotly_plot(df_in=df_both[graph_cols],x_column='settle_date',
                 yaxis2_cols=['fut_spread','opt_spread']))




### Methods to build implied volatilites

In [95]:
def _get_contract_number_from_symbol(symbol):
    c = symbol[0:2]
    if c in ['CL','CB','ES','GE','NG']:
        return 2
    return 2

In [213]:
USE_PYVOL = True
def lam_pyvol(r):
    try:
        return implied_volatility.implied_volatility(r.close_x,r.close_y,r.strike,.02,r.dte/365, r.pc.lower())
    except:
        return -1
# lam_pyvol = lambda r:implied_volatility.implied_volatility(r.close_x,r.close_y,r.strike,.02,r.dte/365, r.pc.lower())
lam_mibian = lambda r:mibian.BS([r.close_y,r.strike,2,r.dte], callPrice=r.close_x).impliedVolatility

def get_implieds(df,df_expiry_dates,contract,contract_num=2):
    df2 = df[['symbol','contract_num','pc','settle_date','strike','close_x','close_y']]
    df2 = df2[(((df2.pc=='C' )& (df2.strike>=df2.close_y)) | ((df2.pc=='P' ) & (df2.strike<df2.close_y)))  & (df2.symbol.str.contains(contract))]
    cnum = _get_contract_number_from_symbol(contract)
    if contract_num is not None:
        df2 = df2[df2.contract_num==contract_num]
    if len(df2)<1:
        return None
    phigh = df2.close_y.max()
    plow = df2.close_y.min()
    high_strike = round(phigh * 1.3)
    low_strike = round(plow * .7)
    df2 = df2[(df2.strike>=low_strike) & (df2.strike<=high_strike)]

    df9 = df2[df2.symbol==contract]
    df9 = df9.merge(df_expiry_dates.rename(columns={'settle_date':'expiry'}),on='symbol',how='inner')
    df9['syear'] = df9.settle_date.astype(str).str.slice(0,4).astype(int)
    df9['smon'] = df9.settle_date.astype(str).str.slice(4,6).astype(int)
    df9['sday'] = df9.settle_date.astype(str).str.slice(6,8).astype(int)
    df9['eyear'] = df9.expiry.astype(str).str.slice(0,4).astype(int)
    df9['emon'] = df9.expiry.astype(str).str.slice(4,6).astype(int)
    df9['eday'] = df9.expiry.astype(str).str.slice(6,8).astype(int)
    df9['sdatetime'] = df9.apply(lambda r:datetime.datetime(r.syear,r.smon,r.sday),axis=1)
    df9['edatetime'] = df9.apply(lambda r:datetime.datetime(r.eyear,r.emon,r.eday),axis=1)
    df9['dte'] = df9.edatetime - df9.sdatetime
    df9.dte = df9.dte.dt.days
    df9 = df9[['symbol','settle_date','pc','contract_num','strike','close_x','close_y','dte']]
    df10 = df9.iloc[:len(df9)].copy()
    df10.index = list(range(len(df10)))
    if USE_PYVOL:
        df10['iv'] = df10.apply(lam_pyvol,axis=1)
    else:
        n = 100
        for i in tqdm_notebook(np.arange(0,len(df10)-n,n)):
                df10.loc[i:i+n,'iv'] = df10.loc[i:i+n].apply(lam_mibian,axis=1)
        print(f'doing remaining {datetime.datetime.now()}')
        i = df10[df10.iv.isna()].index[0]
        df10.loc[i:,'iv'] = df10.loc[i:].apply(lam_mbian,axis=1)
        print(f'done with remaining {datetime.datetime.now()}')
    return df10



#### example of using mibian for options calcs (we use py_vollib instead)

In [97]:
def _test_mibian():
    underlying=1.4565
    strike=1.45
    interest = 1
    days=30
    opt_info = [underlying,strike,interest,days]
    c = mibian.BS(opt_info, volatility=20)
    print(c.callPrice,
    c.putPrice,
    c.callDelta,
    c.putDelta,
    c.callDelta2,
    c.putDelta2,
    c.callTheta,
    c.putTheta,
    c.callRho,
    c.putRho,
    c.vega,
    c.gamma)


    co = mibian.BS(opt_info, callPrice=c.callPrice)
    co.impliedVolatility

In [99]:
_test_mibian()

0.03721154839277063 0.029520257209636247 0.5481584196590104 -0.4518415803409897 -0.52495254471764 0.4742258751560606 -0.0005720853891507385 -0.0005323919998680845 0.000625628375211434 -0.0005651733032681817 0.0016536933299310715 4.742153534821618


#### Show simple example of using py_vol package

In [101]:
def _test_py_vollib():
    #CL,Q2019,560P,07/02/2019,0.6,1.61,0.54,1.54,1997,4465
    F = 56.25
    K = 56
    sigma = .366591539
    flag = 'p'
    t = 15/365.0
    r = .025
    discounted_call_price = black.black(flag, F, K, t, r, sigma)
    dcp = 1.54
    ivpy = implied_volatility.implied_volatility(dcp, F, K, r, t, flag)
    ivmn = mibian.BS([F,K,2.5,15], callPrice=dcp).impliedVolatility
    return discounted_call_price,ivpy,ivmn


In [102]:
_test_py_vollib()

(1.5399999992892466, 0.36659153915713766, 30.517578125)

### Method to get closest-to-the-money and at-the-money (atm) vols

In [169]:
def get_closest_to_the_money_vols(df_implied_vols,
                 opt_close_col='close_x',
                 fut_close_col = 'close_y',
                 iv_col = 'iv'):
    df_in = df_implied_vols.copy()
    df_in['amt_away_from_money'] = df_in.apply(lambda r: abs(r[fut_close_col]-r.strike),axis=1)
    gb_cols = ['settle_date','symbol']
    df_min_amt_away = df_in[gb_cols + ['amt_away_from_money']].groupby(gb_cols,as_index=False).min()
    df_ret = df_min_amt_away.merge(
        df_in,on=gb_cols + ['amt_away_from_money'],how='inner')
    df_lowest_iv = df_ret[['settle_date','symbol','iv']].groupby(
        ['settle_date','symbol'],
        as_index=False).min()
    df_ret = df_ret.merge(df_lowest_iv,on=['settle_date','symbol','iv'],how='inner')
    return df_ret


In [174]:
import pdb

def get_even_moneyness_strikes(df10):
    # define amounts around the money which will help create strikes to add
    moneyness = np.arange(.7,1.4,.05).round(6)
    # define columns on which to execute groupby
#     gb_cols = ['symbol','settle_date','pc','contract_num','dte','close_y']
    gb_cols = ['symbol','settle_date','contract_num','dte','close_y']
    # define function used in groupby.apply to create strikes and iv's at those strikes
    #   where the strikes are an even amount from the money 
    #   (like .7, .8, ... 1, 1.1, 1.2, etc)
    def _add_even_moneyness_strikes(df):
        # get underlying from first row (the groupby makes them all the same)
        r = df.iloc[0]
        underlying = r.close_y
        # create new rows to append to df, using only the gb_cols
        df_ret1 = df.iloc[:len(moneyness)][gb_cols].copy()
        # add nan iv's !!!! MUST BE np.nan - NOT None
        df_ret1['iv'] = np.nan
        # add new strikes
        df_ret1['strike'] = moneyness * underlying
        # append the new strikes
        try:
            # try with using the sort=True options for versions of pandas after 0.23
            dfa = df.append(df_ret1,ignore_index=True,sort=True).copy()
        except:
            # otherwise do not specify sort
            dfa = df.append(df_ret1,ignore_index=True).copy()
        df_ret2 = dfa.sort_values(['symbol','settle_date','pc','strike'])
        df_ret2 = df_ret2.drop_duplicates(subset='strike')
        # set the index to the strike so that interpolate works
        df_ret2.index = df_ret2.strike
        # create interpolated iv's
        df_ret2['iv'] = df_ret2.iv.interpolate(method='polynomial', order=2)
        # reset the index
        df_ret2.index = list(range(len(df_ret2)))
        return df_ret2

    # start here
    df11 = df10.groupby(gb_cols).apply(_add_even_moneyness_strikes).copy()
    df11.index = list(range(len(df11)))
    df11['moneyness'] = df11.strike / df11.close_y
    df11.moneyness = df11.moneyness.round(4)

    df12 = df11[(df11.moneyness.isin(moneyness)) & (~df11.iv.isna())].copy()
    df12.moneyness  = df12.moneyness - 1
    df12.index = list(range(len(df12)))
    df12_atm = df12[df12.moneyness==0][['symbol','settle_date','pc','iv']]
    df12_atm = df12_atm.rename(columns={'iv':'atm_iv'})
    
    df12_atm = df12_atm.drop_duplicates()
    df12_atm.pc = ''#np.nan
    df12.pc = ''#np.nan

    df12 = df12.merge(df12_atm,on=['symbol','settle_date','pc'],how='inner')

#     df12 = df12.merge(df12_atm,on=['symbol','settle_date'],how='inner')
#     df12['pc'] = np.nan

    df12.moneyness = df12.moneyness.round(4)
    df12['vol_skew'] = (df12.iv - df12.atm_iv).round(4)
    return df12



### Method to get volatility skews per symbol

In [105]:
def skew_per_symbol(symbol,strike_divisor=None):
    '''
    For a symbol like CLM16 or EZH19, create 2 Dataframes
      1. df_iv - contains rows of implied vols, for only the 'pseudo' strikes that are an even
                 percent away from the money for each settle_date
      2. df_skew - contains one row per day of skew data of for 'pseudo' strikes that are an even
                 percent away from the money for each settle_date
    '''
    _exception = None
    _stacktrace = None
    df_iv = None
    df_skew = None
    try:
        df,df_expiry_dates = get_postgres_data(symbol)
        if len(df[df.contract_num==2])>0:
            df10 = get_implieds(df,df_expiry_dates,symbol)
            df12 = get_even_moneyness_strikes(df10)
            df_sk = create_skew_per_date_df(df12)
            df_sk.index = list(range(len(df_sk)))
            df_skt = df_sk.T
            df_skt.columns = df_skt.loc['moneyness']
            df_skt = df_skt.iloc[1:].copy()
            df_skt['symbol'] = symbol
            df_skt['settle_date'] = df_skt.index
            df_iv = df12.copy() 
            df_skew = df_skt.copy()
    except Exception as e:
        _exception = str(e)
        _stacktrace = traceback.format_exc()
    return df_iv,df_skew,_exception,_stacktrace

### test getting implieds for a single contract

In [216]:
strike_div = None if SYMBOL_TO_RESEARCH not in STRIKE_DIVISORS.keys() else STRIKE_DIVISORS[SYMBOL_TO_RESEARCH]
sym = "ESH11"
df,df_expiry_dates = get_postgres_data(sym)
df_implieds = get_implieds(df,df_expiry_dates,sym)
df_vols = None
df_ctm_vols = None
df_atm_money = None
if df_implieds is not None:
    df_ctm_vols = get_closest_to_the_money_vols(df_implieds)
    df_even_money = get_even_moneyness_strikes(df_implieds)
    df_atm_money = df_even_money[df_even_money.moneyness==0]
    df_vols = df_ctm_vols.merge(
        df_atm_money[['settle_date','symbol','atm_iv']],
        on=['settle_date','symbol'],how='inner')
else:
    print(f'no implied vols for symbol: {sym}')

no implied vols for symbol: ESH11


In [210]:
df,df_expiry_dates = get_postgres_data('ESH11')
df_implieds = get_implieds(df,df_expiry_dates,'ESH11')


ValueError: cannot convert float NaN to integer

In [203]:
df_vols

,settle_date,symbol,amt_away_from_money,pc,contract_num,strike,close_x,close_y,dte,iv,atm_iv
0,20150921,ESH16,0.50,P,2,1955.0,101.00,1955.50,179,0.187311,0.187198
1,20150922,ESH16,0.50,C,2,1925.0,105.25,1924.50,178,0.198828,0.198949
2,20150923,ESH16,1.00,P,2,1920.0,103.00,1921.00,177,0.196021,0.195780
3,20150924,ESH16,0.75,P,2,1910.0,105.50,1910.75,176,0.202154,0.201974
4,20150925,ESH16,1.25,P,2,1910.0,105.25,1911.25,175,0.202687,0.202375
...,...,...,...,...,...,...,...,...,...,...,...
59,20151214,ESH16,0.50,C,2,2010.0,76.00,2009.50,95,0.187452,0.187592
60,20151215,ESH16,2.00,P,2,2035.0,71.75,2037.00,94,0.177441,0.176852
61,20151216,ESH16,1.25,C,2,2065.0,66.75,2063.75,93,0.162933,0.163388
62,20151217,ESH16,0.25,C,2,2025.0,70.50,2024.75,92,0.175076,0.175169


In [204]:
iplot(plotly_plot(
    df_in=df_vols[['settle_date','close_y','atm_iv']],
    x_column='settle_date',yaxis2_cols=['atm_iv']))

In [196]:
sql_all_contracts = f"""
select distinct symbol from {opttab} 
where substring(symbol,1,2) = 'ES' and
substring(symbol,3,1) in ('H','M','U','Z')
"""
all_contracts = sorted(pga.get_sql(sql_all_contracts).symbol.values)


In [234]:
all_contracts_reversed = sorted([f'{c[:2]}{c[-2:]}{c[2]}' for c in all_contracts])
all_contracts2 = [f'{c[:2]}{c[-1]}{c[2:4]}' for c in all_contracts_reversed]
all_contracts2

['ESH11',
 'ESM11',
 'ESU11',
 'ESZ11',
 'ESH12',
 'ESM12',
 'ESU12',
 'ESZ12',
 'ESH13',
 'ESM13',
 'ESU13',
 'ESZ13',
 'ESH14',
 'ESM14',
 'ESU14',
 'ESZ14',
 'ESH15',
 'ESM15',
 'ESU15',
 'ESZ15',
 'ESH16',
 'ESM16',
 'ESU16',
 'ESZ16',
 'ESH17',
 'ESM17',
 'ESU17',
 'ESZ17',
 'ESH18',
 'ESM18',
 'ESU18',
 'ESZ18',
 'ESH19',
 'ESM19',
 'ESU19',
 'ESZ19',
 'ESH20',
 'ESM20',
 'ESU20',
 'ESZ20']

In [233]:
sbstrike = .1325
sb = .1172
sbcall = .0006
sbput = sbcall + sbstrike - sb
sbput, sbstrike - sb,sbput + sb

(0.015899999999999997, 0.015300000000000008, 0.1331)

In [235]:
for sym in all_contracts2:
    print(f"graphing symbol: {sym}")
    commod = sym[0:2]
    strike_div = None if commod not in STRIKE_DIVISORS.keys() else STRIKE_DIVISORS[commod]
    df,df_expiry_dates = get_postgres_data(sym)
    if df is None or len(df)<10:
        print(f'no data for symbol: {sym}')
        continue
    df_implieds = get_implieds(df,df_expiry_dates,sym)
    df_vols = None
    df_ctm_vols = None
    df_atm_money = None
    if df_implieds is not None:
        df_ctm_vols = get_closest_to_the_money_vols(df_implieds)
        df_even_money = get_even_moneyness_strikes(df_implieds)
        df_atm_money = df_even_money[df_even_money.moneyness==0]
        df_vols = df_ctm_vols.merge(
            df_atm_money[['settle_date','symbol','atm_iv']],
            on=['settle_date','symbol'],how='inner')
    else:
        print(f'no implied vols for symbol: {sym}')
        continue
    df_ctm_vols = get_closest_to_the_money_vols(df_implieds)
    df_even_money = get_even_moneyness_strikes(df_implieds)
    df_atm_money = df_even_money[df_even_money.moneyness==0]
    df_vols = df_ctm_vols.merge(
        df_atm_money[['settle_date','symbol','atm_iv']],
        on=['settle_date','symbol'],how='inner')
    iplot(plotly_plot(
        df_in=df_vols[['settle_date','close_y','atm_iv']],
        x_column='settle_date',yaxis2_cols=['atm_iv']))    

graphing symbol: ESH11
no implied vols for symbol: ESH11
graphing symbol: ESM11


graphing symbol: ESU11


graphing symbol: ESZ11


graphing symbol: ESH12


graphing symbol: ESM12


...

graphing symbol: ESU19


graphing symbol: ESZ19


graphing symbol: ESH20


graphing symbol: ESM20


graphing symbol: ESU20
no implied vols for symbol: ESU20
graphing symbol: ESZ20
no implied vols for symbol: ESZ20


## END